# 🛡️ ChildSafetyOS - Technical Documentation

### AI-Powered Parental Control System for Android

---

**Project Type:** Android Application + Cloud Backend + Web Dashboard  
**Tech Stack:** Kotlin, Jetpack Compose, TensorFlow Lite, Firebase, Node.js  
**Target SDK:** Android 14 (API 36)  
**Documentation Version:** 1.1 | January 2026

---

## 📋 Table of Contents

1. [Project Overview](#section-1-project-overview)
2. [System Architecture](#section-2-system-architecture)
3. [Core Components](#section-3-core-components)
4. [Machine Learning Pipeline](#section-4-machine-learning-pipeline)
5. [Firebase Integration](#section-5-firebase-integration)
6. [Security Analysis](#section-6-security-analysis)
7. [Setup & Installation](#section-7-setup-installation)
8. [Demo Scenarios](#section-8-demo-scenarios)
9. [Future Enhancements](#section-9-future-enhancements)
10. [Audit Report](#section-10-audit-report)

---

# 📖 Section 1: Project Overview

## The Problem

Digital content has become increasingly accessible, but not all content is appropriate. Studies show:
- **93%** of children aged 8-12 have access to smartphones
- **56%** of children have encountered inappropriate content online
- Traditional parental controls are easily bypassed

## Our Solution

**ChildSafetyOS** is a comprehensive, AI-powered content safety system that provides:

| Feature | Description |
|---------|-------------|
| 🌐 **Safe Browser** | Built-in browser with real-time content filtering |
| 🔒 **VPN Protection** | Network-level blocking via Cloudflare Family DNS |
| 🤖 **ML Image Detection** | On-device NSFW classification using TensorFlow Lite |
| 🚫 **App Lock** | Prevents access to Settings & Play Store |
| 📊 **Parent Dashboard** | Real-time monitoring via Firebase & web dashboard |
| 📧 **Instant Alerts** | Email notifications for critical content detection |
| 🎯 **Age Mode Selector** | Child/Teen/Adult modes with appropriate thresholds |

## Key Differentiators

1. **On-Device ML**: All image classification happens locally - no images sent to cloud
2. **Multi-Layer Defense**: 5 independent filtering layers for maximum protection
3. **Tamper-Resistant**: Device Admin + Accessibility Service prevents uninstallation
4. **Privacy-First**: DPDP Act compliant with email-based data deletion request
5. **Universal Protection**: Even Adult mode blocks highly explicit content

## Philosophy

> **⚠️ Important:** Explicit content can be harmful, addictive, or triggering at ANY age. This is NOT about restricting rights - it's about promoting digital wellbeing. Even Adult mode maintains protection against highly explicit pornographic content.

---

# 🏗️ Section 2: System Architecture

## High-Level Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           ChildSafetyOS Architecture                         │
└─────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                              ANDROID APPLICATION                              │
├─────────────────┬─────────────────┬────────────────┬─────────────────────────┤
│                 │                 │                │                         │
│  MainActivity   │ SafeBrowser     │  VPN Service   │   AccessibilityService  │
│  (Compose UI)   │ Activity        │                │   (App Lock)            │
│                 │                 │                │                         │
│  • VPN Toggle   │ • WebView       │ • DNS Filter   │  • Block Settings       │
│  • Age Selector │ • JS Bridge     │ • Packet Route │  • Block Play Store     │
│  • Admin Setup  │ • Image Scanner │ • Cloudflare   │  • PIN Lock Screen      │
│                 │                 │                │                         │
└────────┬────────┴────────┬────────┴───────┬────────┴────────────┬────────────┘
         │                 │                │                     │
         ▼                 ▼                ▼                     ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                            POLICY & AI ENGINE                                │
├───────────────────┬───────────────────┬──────────────────────────────────────┤
│                   │                   │                                      │
│   PolicyEngine    │   DomainPolicy    │         TfLiteManager                │
│                   │                   │                                      │
│ • Domain Check    │ • Adult Sites     │  ┌─────────────────────────────────┐ │
│ • Text Check      │ • Gambling        │  │    nsfw_mobilenet.tflite        │ │
│ • Emoji Check     │ • Drugs/Violence  │  │    text_risk.tflite             │ │
│ • Age Thresholds  │ • Proxy/VPN       │  │    video_frame_risk.tflite      │ │
│                   │ • Social Media    │  └─────────────────────────────────┘ │
└───────────────────┴───────────────────┴──────────────────────────────────────┘
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                             FIREBASE BACKEND                                 │
├────────────────────────────┬────────────────────────────────────────────────┤
│                            │                                                │
│        FIRESTORE           │              CLOUD FUNCTIONS                   │
│                            │                                                │
│  /devices/{deviceId}       │    • sendAlertEmail()                          │
│    /events/{eventId}       │    • Triggers on CRITICAL alerts               │
│    /daily_stats/{date}     │    • Gmail/SendGrid integration                │
│    /alerts/{alertId}       │                                                │
│                            │                                                │
└────────────────────────────┴────────────────────────────────────────────────┘
```

## Content Filtering Pipeline

```
User Request → L1: DNS (Cloudflare 1.1.1.3)
                    ↓ (allowed)
              L2: Domain Policy (100+ blocklist)
                    ↓ (allowed)
              L3: URL Keyword Scan (search queries)
                    ↓ (allowed)
              L4: ML Image Classification (TFLite)
                    ↓ (allowed)
              L5: Text/Emoji Analysis
                    ↓
              ✅ CONTENT DISPLAYED
              
              [Any layer blocks] → 🚫 BLOCKED PAGE SHOWN
```

---

# 🔧 Section 3: Core Components

## 3.1 MainActivity (Launcher)

The main entry point provides a control panel with **Age Mode Selector**.

**File:** `app/src/main/java/com/childsafety/os/ui/MainActivity.kt`

```kotlin
class MainActivity : ComponentActivity() {

    private var vpnEnabled by mutableStateOf(false)
    private var selectedAgeGroup by mutableStateOf(AgeGroup.CHILD)

    // Age Mode Selector UI
    AgeGroup.values().forEach { ageGroup ->
        val emoji = when (ageGroup) {
            AgeGroup.CHILD -> "👶"
            AgeGroup.TEEN -> "🧑"
            AgeGroup.ADULT -> "👤"
        }
        Button(onClick = { selectedAgeGroup = ageGroup }) {
            Text("$emoji ${ageGroup.name}")
        }
    }
    
    // Show description for selected mode
    val description = ThresholdProvider.getDescription(selectedAgeGroup)
    Text(description)
    
    // Adult mode warning
    if (selectedAgeGroup == AgeGroup.ADULT) {
        Text("⚠️ Even in Adult mode, highly explicit content is blocked.")
    }
}
```

### UI Features:
- **VPN Toggle**: Enable/disable network-level filtering
- **Age Mode Selector**: Child 👶 / Teen 🧑 / Adult 👤
- **Uninstall Protection**: Activate Device Admin
- **Settings Lock**: Enable Accessibility Service
- **Data Deletion**: Email request to app owner (DPDP compliant)

## 3.2 Data Deletion (DPDP Compliance)

Instead of a direct delete button, users must request data deletion via email:

```kotlin
// Request data deletion via email
onRequestDataDeletion = {
    val deviceId = ChildSafetyApp.appDeviceId
    val emailIntent = Intent(Intent.ACTION_SENDTO).apply {
        data = Uri.parse("mailto:")
        putExtra(Intent.EXTRA_EMAIL, arrayOf("support@childsafetyos.com"))
        putExtra(Intent.EXTRA_SUBJECT, "[DPDP] Data Deletion Request - Device: $deviceId")
        putExtra(Intent.EXTRA_TEXT, """
            Data Deletion Request (DPDP Act Compliance)
            
            Device ID: $deviceId
            Request Type: Delete All My Data
            
            I hereby request the deletion of all data associated with 
            the above Device ID as per the Digital Personal Data 
            Protection Act.
            
            Please confirm once the data has been deleted.
        """.trimIndent())
    }
    startActivity(Intent.createChooser(emailIntent, "Send request via..."))
}
```

### Why Email Request?
- **Accountability**: Creates audit trail of deletion requests
- **Verification**: Prevents accidental data loss
- **Compliance**: Meets DPDP Act requirements with proper documentation

---

# 🤖 Section 4: Machine Learning Pipeline

## 4.1 TensorFlow Lite Models

Three ML models run entirely on-device for privacy:

| Model | File | Size | Input | Output |
|-------|------|------|-------|--------|
| **NSFW MobileNet** | `nsfw_mobilenet.tflite` | ~5MB | 224×224×3 RGB | 5 classes |
| **Text Risk** | `text_risk.tflite` | ~100KB | 3D vector | Binary (0-1) |
| **Video Frame** | `video_frame_risk.tflite` | ~5MB | Video frames | Risk score |

## 4.2 Age-Appropriate Thresholds

### Philosophy

> **Explicit content can be harmful or triggering at ANY age.** We provide progressively relaxed thresholds, but even Adults have protection from highly explicit pornographic content.

### Threshold Configuration

| Age Group | Porn | Hentai | Sexy | Description |
|-----------|------|--------|------|-------------|
| **CHILD** (Under 13) | 0.15 | 0.15 | 0.25 | Maximum protection. Blocks all suggestive content. |
| **TEEN** (13-17) | 0.35 | 0.35 | 0.50 | Strong protection. Allows mature themes, blocks explicit. |
| **ADULT** (18+) | 0.85 | 0.85 | 0.95 | Relaxed filtering. Still blocks highly explicit content. |

### Why Adults Have Thresholds?

1. **Addiction Prevention**: Pornography can be psychologically addictive
2. **Triggering Content**: Can be harmful for trauma survivors
3. **Illegal Content**: Blocks potentially illegal extreme content
4. **Safety App Philosophy**: This is a SAFETY app, not an unrestricted browser

```kotlin
// ThresholdProvider.kt
AgeGroup.ADULT -> ImageThresholds(
    pornThreshold = 0.85f,    // Block only obvious porn
    hentaiThreshold = 0.85f,  // Block explicit animated
    sexyThreshold = 0.95f     // Allow most suggestive, block explicit
)
```

## 4.3 Domain Blocking by Age

Domain blocking is also age-aware:

| Age Group | Blocked Categories |
|-----------|--------------------|
| CHILD | ALL categories (Adult, Gambling, Drugs, Violence, Proxy, Social Media, Dating) |
| TEEN | All except Social Media (with supervision) |
| ADULT | Adult + Violence only |

```kotlin
// PolicyEngine.kt
fun shouldBlockDomain(host: String?, ageGroup: AgeGroup): Boolean {
    val decision = DomainPolicy.evaluate(host)
    if (!decision.blocked) return false
    
    return when (ageGroup) {
        AgeGroup.CHILD -> true // Block all
        
        AgeGroup.TEEN -> {
            // Allow social media with supervision
            decision.category != BlockCategory.SOCIAL_MEDIA
        }
        
        AgeGroup.ADULT -> {
            // Only block porn and violence
            decision.category == BlockCategory.ADULT ||
            decision.category == BlockCategory.VIOLENCE
        }
    }
}
```

---

# 🔐 Section 6: Security Analysis

## 6.1 Defense-in-Depth Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    LAYER 1: NETWORK (VPN)                       │
│  Cloudflare Family DNS (1.1.1.3)                               │
│  • Blocks adult sites at DNS resolution                        │
│  • Works for ALL apps on device                                │
├─────────────────────────────────────────────────────────────────┤
│                    LAYER 2: DOMAIN BLOCKLIST                    │
│  DomainPolicy with 100+ blocked domains                        │
│  • Age-appropriate blocking per category                       │
│  • Subdomain matching                                          │
├─────────────────────────────────────────────────────────────────┤
│                    LAYER 3: SEARCH KEYWORD FILTER               │
│  URL analysis for explicit search queries                      │
│  • Enforces SafeSearch on all search engines                   │
├─────────────────────────────────────────────────────────────────┤
│                    LAYER 4: ML IMAGE CLASSIFICATION             │
│  TensorFlow Lite NSFW MobileNet                                │
│  • On-device inference (privacy-first)                         │
│  • Age-appropriate thresholds                                  │
├─────────────────────────────────────────────────────────────────┤
│                    LAYER 5: TEXT/EMOJI ANALYSIS                 │
│  Rule-based + ML text classification                           │
│  • Emoji risk scoring with age thresholds                      │
└─────────────────────────────────────────────────────────────────┘
```

## 6.2 Privacy Compliance (DPDP Act)

| Requirement | Implementation |
|-------------|----------------|
| Data Minimization | Only device ID collected |
| Right to Erasure | Email-based deletion request |
| On-Device Processing | ML runs locally |
| Purpose Limitation | Data used only for safety monitoring |

---

# 📋 Section 10: Audit Report

## Summary

| Metric | Value |
|--------|-------|
| Total Kotlin Files | 57 |
| Lines of Code (Kotlin) | ~6,500 |
| ML Models | 3 |
| Blocked Domain Categories | 7 |
| Age Modes | 3 (Child, Teen, Adult) |

## Security Strengths ✅

1. **Multi-Layer Defense**: 5 independent filtering layers
2. **On-Device ML**: Privacy-first, no image uploads
3. **Age-Appropriate Thresholds**: All modes have protection
4. **Tamper-Resistant**: Device Admin + Accessibility protection
5. **DPDP Compliance**: Email-based data deletion requests

## Changes Made in This Version

### 1. Age Mode Selector
- Added UI for selecting Child/Teen/Adult modes
- Each mode has description and appropriate thresholds
- Adult mode shows warning about continued protection

### 2. Universal Protection Philosophy
- Removed "disabled" adult thresholds
- Adults now have 0.85 threshold (blocks obvious porn)
- Rationale: explicit content can be harmful at any age

### 3. Data Deletion via Email
- Replaced direct delete button with email request
- Pre-filled email with device ID and DPDP reference
- Creates audit trail for compliance

---

*Documentation updated: January 2026*